# Import Basic Lib

In [1]:
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pandas.core.common import flatten

from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
import gensim
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.corpus import stopwords
from googletrans import Translator
import random
import sentencepiece
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.utils import shuffle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ariza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ariza\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ariza\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
c:\Users\ariza\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_colwidth', 120)
pd.set_option('display.max_row', None)

# Load File

In [3]:
cols_to_load = ['sentimen','tweet']
df_tweets = pd.read_csv('./tweet.csv', usecols=cols_to_load)
sentiments = df_tweets.sentimen
df_tweets['cleaned_tweets'] = tweets = df_tweets.tweet 

In [4]:
print(df_tweets.describe())
print('-'*100)
print(df_tweets.info())
print('-'*100)
print(df_tweets.shape)
print('-'*100)
sentiments.value_counts()

       sentimen  \
count      1815   
unique        3   
top     positif   
freq        612   

                                                                                                                          tweet  \
count                                                                                                                      1815   
unique                                                                                                                     1815   
top     Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygeru...   
freq                                                                                                                          1   

                                                                                                                 cleaned_tweets  
count                                                                                                                 

positif    612
netral     607
negatif    596
Name: sentimen, dtype: int64

# Text Cleaning

## Tag @

In [5]:
def clean_tag(tweet):
    return tweet.str.replace('@','')

In [6]:
df_tweets['cleaned_tweets'] = clean_tag(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0    Kata prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung...
1                              Batuan Langka, Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy?  http://dlvr.it/R2pvZVÂ 
2    Di era Jokowi, ekonomi Indonesia semakin baik. #01IndonesiaMaju #JokowiLagi #JokowiMenangTotalDebat pic.twitter.com/...
3    Bagi Sumatera Selatan, Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun. Indonesia maju, J...
4    Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat, terus masyarakatnya ngeluh karena negara n...
Name: cleaned_tweets, dtype: object
1810                        Negarawan sejati sll bangga dan mengedepankan harga diri bangsanya yg berdaulat  #2019GantiPresiden
1811                                           1. HRS ceramah di Damai Indonesiaku 2. Perekonomian makin membaik.  #PutihkanGBK
1812    Mari bangun bangsa dgn mendukung perekonomian negara bersama Pak Jokowi. Ay

## HTTPS

In [7]:
def remove_url_and_twitter_pics(tweet):
    # Menghapus URL
    tweets_no_url = tweet.apply(lambda x: re.sub(r'http\S+|www.\S+', '', x, flags=re.MULTILINE))
    # Menghapus link gambar Twitter
    tweets_no_url_no_pics = tweets_no_url.apply(lambda x: re.sub(r'pic.twitter.com/\S+', '', x))
    return tweets_no_url_no_pics

In [8]:
df_tweets['cleaned_tweets'] = remove_url_and_twitter_pics(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0    Kata prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung...
1                                                    Batuan Langka, Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy?   
2                       Di era Jokowi, ekonomi Indonesia semakin baik. #01IndonesiaMaju #JokowiLagi #JokowiMenangTotalDebat 
3    Bagi Sumatera Selatan, Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun. Indonesia maju, J...
4    Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat, terus masyarakatnya ngeluh karena negara n...
Name: cleaned_tweets, dtype: object
1810                        Negarawan sejati sll bangga dan mengedepankan harga diri bangsanya yg berdaulat  #2019GantiPresiden
1811                                           1. HRS ceramah di Damai Indonesiaku 2. Perekonomian makin membaik.  #PutihkanGBK
1812                                  Mari bangun bangsa dgn mendukung perekonomian

## Remove Hashtag

In [9]:
def remove_hashtags(tweet):
    return tweet.apply(lambda x: re.sub(r'#\S+', '', x))

In [10]:
df_tweets['cleaned_tweets'] = remove_hashtags(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0    Kata prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung...
1                                                    Batuan Langka, Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy?   
2                                                                         Di era Jokowi, ekonomi Indonesia semakin baik.    
3    Bagi Sumatera Selatan, Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun. Indonesia maju, J...
4    Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat, terus masyarakatnya ngeluh karena negara n...
Name: cleaned_tweets, dtype: object
1810                                          Negarawan sejati sll bangga dan mengedepankan harga diri bangsanya yg berdaulat  
1811                                                       1. HRS ceramah di Damai Indonesiaku 2. Perekonomian makin membaik.  
1812                                  Mari bangun bangsa dgn mendukung perekonomian

## Special Character or Punctuation

In [11]:
def remove_punctuation(tweet):
    return tweet.apply(lambda x: re.sub(r'\W+', ' ', x))


In [12]:
df_tweets['cleaned_tweets'] = remove_punctuation(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0     Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini ðŸ pasti hoax buatan penguasa ya kan rockygerung ðŸ œ â 
1                                                        Batuan Langka Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy 
2                                                                              Di era Jokowi ekonomi Indonesia semakin baik 
3    Bagi Sumatera Selatan Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18 5 triliun Indonesia maju Joko...
4    Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat terus masyarakatnya ngeluh karena negara ng...
Name: cleaned_tweets, dtype: object
1810                                           Negarawan sejati sll bangga dan mengedepankan harga diri bangsanya yg berdaulat 
1811                                                           1 HRS ceramah di Damai Indonesiaku 2 Perekonomian makin membaik 
1812                                       Mari bangun bangsa dgn mendukung perekon

## Replace Angka Menjadi Kata

In [13]:
def replace_numbers(tweet):
    tweet = tweet.replace('01', 'jokowi')
    tweet = tweet.replace('02', 'prabowo')
    return tweet


In [14]:
df_tweets['cleaned_tweets'] = replace_numbers(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0     Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini ðŸ pasti hoax buatan penguasa ya kan rockygerung ðŸ œ â 
1                                                        Batuan Langka Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy 
2                                                                              Di era Jokowi ekonomi Indonesia semakin baik 
3    Bagi Sumatera Selatan Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18 5 triliun Indonesia maju Joko...
4    Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat terus masyarakatnya ngeluh karena negara ng...
Name: cleaned_tweets, dtype: object
1810                                           Negarawan sejati sll bangga dan mengedepankan harga diri bangsanya yg berdaulat 
1811                                                           1 HRS ceramah di Damai Indonesiaku 2 Perekonomian makin membaik 
1812                                       Mari bangun bangsa dgn mendukung perekon

## Alphanumeric

In [15]:
def filter_alphanumeric(tweet):
    return tweet.apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))

In [16]:
df_tweets['cleaned_tweets'] = filter_alphanumeric(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0           Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini  pasti hoax buatan penguasa ya kan rockygerung    
1                                                        Batuan Langka Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy 
2                                                                              Di era Jokowi ekonomi Indonesia semakin baik 
3    Bagi Sumatera Selatan Asian Games berdampak pd ekonomi langsung diprediksi mencapai   triliun Indonesia maju Jokowi ...
4    Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat terus masyarakatnya ngeluh karena negara ng...
Name: cleaned_tweets, dtype: object
1810                                           Negarawan sejati sll bangga dan mengedepankan harga diri bangsanya yg berdaulat 
1811                                                              HRS ceramah di Damai Indonesiaku  Perekonomian makin membaik 
1812                                        Mari bangun bangsa dgn mendukung pereko

## Lower Case

In [17]:
def to_lowercase(tweet):
    return tweet.str.lower()

In [18]:
df_tweets['cleaned_tweets'] = to_lowercase(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0           kata prabowo indonesia tidak dihargai bangsa asing berita ini  pasti hoax buatan penguasa ya kan rockygerung    
1                                                        batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy 
2                                                                              di era jokowi ekonomi indonesia semakin baik 
3    bagi sumatera selatan asian games berdampak pd ekonomi langsung diprediksi mencapai   triliun indonesia maju jokowi ...
4    negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat terus masyarakatnya ngeluh karena negara ng...
Name: cleaned_tweets, dtype: object
1810                                           negarawan sejati sll bangga dan mengedepankan harga diri bangsanya yg berdaulat 
1811                                                              hrs ceramah di damai indonesiaku  perekonomian makin membaik 
1812                                        mari bangun bangsa dgn mendukung pereko

## Convert Abbreviations

In [19]:
# Data singkatan dari https://github.com/meisaputri21/Indonesian-Twitter-Emotion-Dataset
col_names_abbeviation = ['before','after']
indo_abbreviation = pd.read_csv('./kamus_singkatan.csv', delimiter=';', names=col_names_abbeviation)
indo_abbreviation.head(5)

,before,after
0,aamiin,amin
1,adek,adik
2,adlh,adalah
3,aer,air
4,aiskrim,es krim


In [20]:
def replace_abbreviations(tweet, abbreviation):
    # Buat dictionary dari dataframe indo_abbreviation
    abbreviation_dict = dict(zip(abbreviation['before'], abbreviation['after']))
    
    # Fungsi untuk mengganti kata dalam tweet
    def replace_words(tweet):
        words = tweet.split()
        new_words = [abbreviation_dict[word] if word in abbreviation_dict else word for word in words]
        return ' '.join(new_words)
    
    # Terapkan fungsi replace_words ke setiap tweet
    tweet = tweet.apply(replace_words)
    
    return tweet


In [21]:
# Preprocessing data singkatan pada kolom 'tweet'
print(df_tweets['cleaned_tweets'].head(5))
df_tweets['cleaned_tweets'] = replace_abbreviations(tweet=df_tweets['cleaned_tweets'], abbreviation=indo_abbreviation)
print(df_tweets['cleaned_tweets'].head(5))

0           kata prabowo indonesia tidak dihargai bangsa asing berita ini  pasti hoax buatan penguasa ya kan rockygerung    
1                                                        batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy 
2                                                                              di era jokowi ekonomi indonesia semakin baik 
3    bagi sumatera selatan asian games berdampak pd ekonomi langsung diprediksi mencapai   triliun indonesia maju jokowi ...
4    negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat terus masyarakatnya ngeluh karena negara ng...
Name: cleaned_tweets, dtype: object
0                kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti hoax buatan penguasa ya kan rockygerung
1                                                         batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy
2                                                                               di era jo

## Convert Slank Words

In [22]:
# Data singkatan dari https://github.com/meisaputri21/Indonesian-Twitter-Emotion-Dataset
col_names_slank_words = ['before','after']
indo_slank_words = pd.read_csv('./kamusalay.csv', delimiter=',', names=col_names_slank_words)
indo_slank_words.head(5)

,before,after
0,ajh,saja
1,ajj,saja
2,akoh,aku
3,amaca,ah masa
4,amuh,kamu


In [23]:
def replace_slank_words(tweet, slank_words):
    # Buat dictionary dari dataframe indo_abbreviation
    abbreviation_dict = dict(zip(slank_words['before'], slank_words['after']))
    
    # Fungsi untuk mengganti kata dalam tweet
    def replace_words(tweet):
        words = tweet.split()
        new_words = [abbreviation_dict[word] if word in abbreviation_dict else word for word in words]
        return ' '.join(new_words)
    
    # Terapkan fungsi replace_words ke setiap tweet
    tweet = tweet.apply(replace_words)
    
    return tweet


In [24]:
# Preprocessing data singkatan pada kolom 'tweet'
print(df_tweets['cleaned_tweets'].head(5))
df_tweets['cleaned_tweets'] = replace_slank_words(tweet=df_tweets['cleaned_tweets'], slank_words=indo_slank_words)
print(df_tweets['cleaned_tweets'].head(5))

0                kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti hoax buatan penguasa ya kan rockygerung
1                                                         batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy
2                                                                               di era jokowi ekonomi indonesia semakin baik
3    bagi sumatera selatan asian games berdampak pada  ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi...
4    negara kita ngutang buat bangun  infrastruktur yang sudah dipakai  masyarakat terus masyarakatnya keluh  karena nega...
Name: cleaned_tweets, dtype: object
0                kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti hoax buatan penguasa ya kan rockygerung
1                                                         batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy
2                                                                               di era jo

## Remove WhiteSpace

In [25]:
def remove_spaces(tweet):
    # Menghapus spasi di depan dan belakang kalimat
    tweets_no_spaces = tweet.str.strip()
    # Menghapus spasi berlebih di tengah kalimat
    tweets_no_extra_spaces = tweets_no_spaces.apply(lambda x: re.sub(' +', ' ', x))
    return tweets_no_extra_spaces


In [26]:
df_tweets['cleaned_tweets'] = remove_spaces(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0                kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti hoax buatan penguasa ya kan rockygerung
1                                                         batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy
2                                                                               di era jokowi ekonomi indonesia semakin baik
3    bagi sumatera selatan asian games berdampak pada ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi ...
4    negara kita ngutang buat bangun infrastruktur yang sudah dipakai masyarakat terus masyarakatnya keluh karena negara ...
Name: cleaned_tweets, dtype: object
1810                                          negarawan sejati sll bangga dan mengedepankan harga diri bangsanya yang berdaulat
1811                                                              harus ceramah di damai indonesiaku perekonomian makin membaik
1812                                       mari bangun bangsa dengan mendukung pere

## Stopwords

### Menggunakan NLTK Stopwords Indonesia

In [27]:
def remove_stopwords(tweet):
    # get stopwords dictionary
    stop_words = set(stopwords.words('indonesian'))

    # remove stopwords from each element of the series
    cleaned_series = tweet.apply(lambda x: ' '.join([word for word in x.lower().split(' ') if word not in stop_words]))

    return cleaned_series


In [28]:
df_tweets['cleaned_tweets'] = remove_stopwords(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0                                         prabowo indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung
1                                                              batuan langka tasbih jokowi hadiah habib luthfi seharga mercy
2                                                                                               era jokowi ekonomi indonesia
3            sumatera selatan asian games berdampak ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat
4    negara ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh negara ngutang negara ngutang utang dibay...
Name: cleaned_tweets, dtype: object
1810            negarawan sejati sll bangga mengedepankan harga bangsanya berdaulat
1811                                 ceramah damai indonesiaku perekonomian membaik
1812              mari bangun bangsa mendukung perekonomian negara jokowi ayo kerja
1813                                   bantu majukan perekonomian bangsa jokowi yuk
181

### Buat Kamus Stopwords

In [29]:
 # filter kata kurang dari kurang dan sama dengan 3 huruf dan hanya huruf
def filter_short_words(tweet):
    return tweet.apply(lambda x: ' '.join([word for word in x.split() if (len(word) <= 3 and word.isalpha() and word.strip() != '')]))

In [30]:
short_words = filter_short_words(df_tweets['cleaned_tweets'])

In [31]:
# buang row yang cuma len = 0 atau ada spasinya saja
short_words = short_words[short_words.str.len() != 0]
print(short_words.head(5))
print(type(short_words))


0     ya
2    era
4     ya
5      e
7    tps
Name: cleaned_tweets, dtype: object
<class 'pandas.core.series.Series'>


In [32]:
# fungsi mengambil kata kata unique
def get_unique_words(short_words):
    # Menggabungkan semua kata dalam series menjadi satu string panjang
    all_words = ' '.join(short_words)
    
    # Memisahkan string menjadi list kata-kata
    words_list = all_words.split()
    
    # Menghapus kata-kata yang sama dari list
    unique_words = list(set(words_list))
    
    return unique_words


In [33]:
short_words = get_unique_words(short_words)


In [34]:
print(short_words)

['sa', 'oke', 'neh', 'qu', 'btw', 'ci', 'pro', 'trm', 'pam', 'egp', 'set', 'ur', 'idn', 'a', 'oh', 'dbs', 'shi', 'va', 'afa', 'cr', 'adi', 'for', 'pqk', 'pas', 'pk', 'eko', 'sah', 'maa', 'aga', 'hp', 'cuy', 'ehh', 'his', 'pc', 'ala', 'kjp', 'ari', 'at', 'prc', 'c', 'rw', 'dwi', 'jas', 'ole', 'usa', 'ahy', 'ika', 'wni', 'yra', 'ego', 'doi', 'n', 'mrt', 'tni', 'dpr', 'woi', 'sll', 'mhn', 'mu', 'gps', 'asn', 'see', 'tik', 'asa', 'har', 'com', 'ml', 'sys', 'ril', 'say', 'yah', 'uas', 'kue', 'do', 'is', 'tdl', 'nb', 'die', 'ngr', 'duo', 'kpu', 'y', 'tul', 'b', 'mba', 'ben', 'tkn', 'smp', 'yuk', 'dk', 'mah', 'dp', 'kpk', 'dim', 'unk', 'tol', 'mal', 'du', 'kas', 'xx', 'ovo', 'ah', 'by', 'th', 'sia', 'rt', 'we', 'top', 'yth', 'den', 'zon', 'sik', 't', 'ham', 'bro', 'hd', 'tgb', 'in', 'cc', 'nyx', 'lae', 'cad', 'sir', 'hak', 'hhh', 'api', 'men', 'nge', 'isu', 'air', 'pbb', 'pki', 'h', 'cs', 'jos', 'krj', 'abg', 'ayo', 'doa', 'shg', 'bhk', 'akn', 'mlh', 'aa', 'via', 'ka', 'fee', 'oom', 'ora', 'l

In [35]:
# List kata-kata yang ingin dihapus
remove_words = ['g', 'ga', 'gak', 'tdk', 'y', 'ya', 'iya','ayo']

# Menghapus kata-kata dari list
short_words = [word for word in short_words if word not in remove_words]
'ga' and 'ya' in short_words

False

In [36]:
# Ubah list menjadi DataFrame
df_short_words = pd.DataFrame(short_words, columns=['short_words'])

# Simpan DataFrame ke dalam file CSV
df_short_words.to_csv('short_word.csv', index=False)

### Buat Stopwords

In [37]:
stopwords_short_words = short_words

In [38]:
def remove_stopwords_short_words(tweet, stopwords):
    # Pisahkan teks dalam setiap baris menjadi kata-kata
    tweet_words = tweet.apply(lambda x: x.split() if isinstance(x, str) else x)

    # Hapus stopword dari teks
    filtered_words = tweet_words.apply(lambda words: [word for word in words if word not in stopwords])

    # Gabungkan kata-kata yang sudah difilter dalam setiap baris menjadi satu string
    filtered_text = filtered_words.apply(' '.join)

    return filtered_text


In [39]:
df_tweets['cleaned_tweets'] = remove_stopwords_short_words(df_tweets['cleaned_tweets'], stopwords_short_words)
df_tweets['cleaned_tweets'].head(5)

0                                         prabowo indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung
1                                                              batuan langka tasbih jokowi hadiah habib luthfi seharga mercy
2                                                                                                   jokowi ekonomi indonesia
3            sumatera selatan asian games berdampak ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat
4    negara ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh negara ngutang negara ngutang utang dibay...
Name: cleaned_tweets, dtype: object

# Text Augmentation

## Texts Backtranslation Augmentation

In [40]:
def translate_tweets(tweets, target_language):
    translator = Translator()

    translated_tweets = []

    for text in tweets:
        try:
            translation = translator.translate(text, dest=target_language)
            translated_tweets.append(translation.text)
        except Exception as e:
            print(f"An error occurred while translating the text: {text}. Error: {e}")
            translated_tweets.append(text)  # Append original text if translation fails

    return translated_tweets



In [41]:
teks = ['saya ingin jajan mobil','saya suka sekali sama kamu']
terjemahan = translate_tweets(teks, 'en')
print(terjemahan)

['I want a car snack', 'I love you the same']


In [42]:
# Translate the 'cleaned_tweets' column
# df_tweets['translated_tweet_en'] = translate_tweets(df_tweets['cleaned_tweets'], 'en')
# df_tweets['translated_tweet_en']

In [43]:
# display(df_tweets['cleaned_tweets'][1])
# display(df_tweets['translated_tweet_en'][1])

In [44]:
# df_tweets['translated_tweet_id'] = translate_tweets(df_tweets['translated_tweet_en'], 'id')
# df_tweets['translated_tweet_id']

In [45]:
# display(df_tweets['cleaned_tweets'][0])
# display(df_tweets['translated_tweet_en'][0])
# display(df_tweets['translated_tweet_id'][0])

## Synonim Replacements

In [46]:
# Function for synonym replacement using WordNet
def synonym_replacement(text, probability=0.1):
    words = word_tokenize(text)
    augmented_words = []

    for word in words:
        if random.uniform(0, 1) < probability:
            synonyms = wordnet.synsets(word, lang='ind')
            if synonyms:
                synonym = synonyms[0].lemmas(lang='ind')[0].name()
                augmented_words.append(synonym)
            else:
                augmented_words.append(word)
        else:
            augmented_words.append(word)

    return ' '.join(augmented_words)

def style_transfer(text, model, tokenizer):
    # Specify the desired style transfer task using a prefix
    input_text = "change the style: " + text

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the style-transferred text
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)

    # Decode and return the transformed text
    style_transferred_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return style_transferred_text

def text_paraphrasing(text, model, tokenizer):
    # Specify the text paraphrasing task using a prefix
    input_text = "paraphrase: " + text

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the paraphrased text
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)

    # Decode and return the paraphrased text
    paraphrased_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return paraphrased_text

def sentence_shuffling(text, probability=0.1):
    sentences = sent_tokenize(text)

    if random.uniform(0, 1) < probability:
        random.shuffle(sentences)
        return ' '.join(sentences)
    else:
        return text

def sentence_deletion(text, probability=0.1):
    sentences = sent_tokenize(text)

    if random.uniform(0, 1) < probability and len(sentences) > 1:
        # Delete a random sentence
        sentences.pop(random.randint(0, len(sentences) - 1))
        return ' '.join(sentences)
    else:
        return text

def word_insertion(text, probability=0.1):
    words = word_tokenize(text)

    if random.uniform(0, 1) < probability:
        # Insert a random word at a random position
        word_to_insert = "merdeka"
        position_to_insert = random.randint(0, len(words))
        words.insert(position_to_insert, word_to_insert)
        return ' '.join(words)
    else:
        return text

# Load the T5 model and tokenizer for style transfer and text paraphrasing
# model_style_transfer = T5ForConditionalGeneration.from_pretrained("t5-small")
# tokenizer_style_transfer = T5Tokenizer.from_pretrained("t5-small")

# Function to perform text augmentation using various techniques
def augment_text_list(texts, augmentation_functions, probabilities):
    augmented_texts = []

    for text in texts:
        augmented_text = text

        for func, prob in zip(augmentation_functions, probabilities):
            augmented_text = func(augmented_text, probability=prob)

        augmented_texts.append(augmented_text)

    return augmented_texts

In [47]:
tweet_augmentation = []

# Define augmentation functions and their corresponding probabilities
augmentation_functions = [synonym_replacement, sentence_shuffling, word_insertion,]
probabilities = [0.3, 0.2, 0.1,]
# tweet_synm_replc = synonym_replacement(tweet_stemmed)
df_tweets['augmented_tweets'] = augment_text_list(df_tweets['cleaned_tweets'], augmentation_functions, probabilities)

In [48]:
display(df_tweets['cleaned_tweets'][:10])
print('-'*50)
display(df_tweets['augmented_tweets'][:10])
print(len(df_tweets['augmented_tweets']))

0                                         prabowo indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung
1                                                              batuan langka tasbih jokowi hadiah habib luthfi seharga mercy
2                                                                                                   jokowi ekonomi indonesia
3            sumatera selatan asian games berdampak ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat
4    negara ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh negara ngutang negara ngutang utang dibay...
5    bisikin jokowi memikirkan perputaran duit golongan sports sekian penentu pertumbuhan perekonomian indonesia fundamen...
6                                                                                            tenang ngoceh jokowi harga mati
7                                                         kerjasa ekonomi bilateral multilateral doakan dengarkan suara hati


--------------------------------------------------


0                                         prabowo Indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung
1                                                              batuan langka tasbih jokowi hadiah habib luthfi seharga mercy
2                                                                                                   jokowi ekonomi Indonesia
3         sumatera selatan asian games berdampak ekonomi langsung diprediksi berhasil triliun indonesia maju jokowi berkuasa
4    keadaan ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh keadaan ngutang negara ngutang budi diba...
5    bisikin jokowi memikirkan kisaran duit golongan sports sekian penentu pertumbuhan perekonomian indonesia asas prabow...
6                                                                                        anteng ngoceh jokowi harga berakhir
7                                                         kerjasa ekonomi bilateral multilateral doakan dengarkan suara hati


1815


In [49]:
tweet_augmented_cleaned = []

  # Looping setiap kalimat
for sentence in df_tweets['augmented_tweets']:

    sentence = re.sub(r"(pic.twitter.com/[^\s]+)", '', sentence) # remove twitter image link

    sentence = re.sub(r'http[s]?\://\S+', "", sentence) # remove url

    sentence = re.sub(r'\S*.com.*\S', "", sentence)# remove url

    sentence = re.sub(r'\S*.co.id.*\S', "", sentence) # remove url

    sentence = re.sub(r'([a-z])([A-Z])', r'\1 \2', sentence) # remove kata kata berdempet

    sentence = re.sub(r"#\w+", "", sentence) #remove hashtag

    sentence = sentence.replace('@', '') # remove at@

    sentence = re.sub(r'\b\d+\b', '', sentence) # remove angka berdempet dg kata

    sentence = re.sub('\d', '', sentence) # remove angka

    # sentence = re.sub(r'\b(\w+)\s+\1\b', r'\1', sentence) # Remove reapeating char

    sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence) # remove punctuation and special character

    sentence = re.sub(r"\s+", " ", sentence) # remove double space

    # Remove repeating words each row
    words = sentence.split()
    sentence = ' '.join(sorted(set(words), key=words.index))

    sentence = sentence.lower() # case folding

    sentence = sentence.strip()

    tweet_augmented_cleaned.append(sentence)

In [50]:
df_tweets['cleaned_augmented_tweets'] = pd.Series(tweet_augmented_cleaned)
df_tweets['cleaned_augmented_tweets']

0                                            prabowo indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung
1                                                                 batuan langka tasbih jokowi hadiah habib luthfi seharga mercy
2                                                                                                      jokowi ekonomi indonesia
3            sumatera selatan asian games berdampak ekonomi langsung diprediksi berhasil triliun indonesia maju jokowi berkuasa
4          keadaan ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh negara budi dibayar penghasilan ya pajak
5       bisikin jokowi memikirkan kisaran duit golongan sports sekian penentu pertumbuhan perekonomian indonesia asas prabow...
6                                                                                           anteng ngoceh jokowi harga berakhir
7                                                            kerjasa ekonomi bilateral multilateral doak

In [51]:
print(df_tweets['cleaned_augmented_tweets'][1])
print(df_tweets['augmented_tweets'][1])

batuan langka tasbih jokowi hadiah habib luthfi seharga mercy
batuan langka tasbih jokowi hadiah habib luthfi seharga mercy


In [52]:
df_tweets.head(5)

,sentimen,tweet,cleaned_tweets,augmented_tweets,cleaned_augmented_tweets
0,negatif,"Kata @prabowo Indonesia tidak dihargai bangsa asing! Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygeru...",prabowo indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung,prabowo Indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung,prabowo indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy? http://dlvr.it/R2pvZVÂ",batuan langka tasbih jokowi hadiah habib luthfi seharga mercy,batuan langka tasbih jokowi hadiah habib luthfi seharga mercy,batuan langka tasbih jokowi hadiah habib luthfi seharga mercy
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik. #01IndonesiaMaju #JokowiLagi #JokowiMenangTotalDebat pic.twitter.com/...",jokowi ekonomi indonesia,jokowi ekonomi Indonesia,jokowi ekonomi indonesia
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun. Indonesia maju, J...",sumatera selatan asian games berdampak ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat,sumatera selatan asian games berdampak ekonomi langsung diprediksi berhasil triliun indonesia maju jokowi berkuasa,sumatera selatan asian games berdampak ekonomi langsung diprediksi berhasil triliun indonesia maju jokowi berkuasa
4,negatif,"Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat, terus masyarakatnya ngeluh karena negara n...",negara ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh negara ngutang negara ngutang utang dibay...,keadaan ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh keadaan ngutang negara ngutang budi diba...,keadaan ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh negara budi dibayar penghasilan ya pajak


In [53]:
# # Untuk save data real dan data augmented dengan cara menumpuk satu per satu row bergantian
# dataset_augmented = pd.DataFrame(columns=['sentimen', 'tweet'])

# for  real_sentimen, real_tweet, augmntd_sentimen, augmntd_tweet in zip(df_tweets['sentimen'], df_tweets['cleaned_tweet'], df_tweets['sentimen'], df_tweets['cleaned_augmented_tweet']):
#   dataset_augmented = dataset_augmented.append({'sentimen': real_sentimen, 'tweet': real_tweet}, ignore_index=True)
#   dataset_augmented = dataset_augmented.append({'sentimen': augmntd_sentimen, 'tweet': augmntd_tweet}, ignore_index=True)

# dataset_augmented

In [54]:
df_tweets_real = df_tweets[['sentimen','cleaned_tweets']]
df_tweets_augmented = df_tweets[['sentimen','cleaned_augmented_tweets']]
df_tweets_real = df_tweets_real.rename(columns={'cleaned_tweets': 'tweet'})
df_tweets_augmented = df_tweets_augmented.rename(columns={'cleaned_augmented_tweets': 'tweet'})
display(df_tweets_real.head(5))
display(df_tweets_augmented.head(5))

,sentimen,tweet
0,negatif,prabowo indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung
1,netral,batuan langka tasbih jokowi hadiah habib luthfi seharga mercy
2,netral,jokowi ekonomi indonesia
3,positif,sumatera selatan asian games berdampak ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat
4,negatif,negara ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh negara ngutang negara ngutang utang dibay...


,sentimen,tweet
0,negatif,prabowo indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung
1,netral,batuan langka tasbih jokowi hadiah habib luthfi seharga mercy
2,netral,jokowi ekonomi indonesia
3,positif,sumatera selatan asian games berdampak ekonomi langsung diprediksi berhasil triliun indonesia maju jokowi berkuasa
4,negatif,keadaan ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh negara budi dibayar penghasilan ya pajak


In [55]:
df_combine_tweets = pd.concat([df_tweets_real, df_tweets_augmented], ignore_index=False)
df_combine_tweets.head(5), df_combine_tweets.shape

(  sentimen  \
 0  negatif   
 1   netral   
 2   netral   
 3  positif   
 4  negatif   
 
                                                                                                                      tweet  
 0                                       prabowo indonesia dihargai bangsa asing berita hoax buatan penguasa ya rockygerung  
 1                                                            batuan langka tasbih jokowi hadiah habib luthfi seharga mercy  
 2                                                                                                 jokowi ekonomi indonesia  
 3          sumatera selatan asian games berdampak ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat  
 4  negara ngutang bangun infrastruktur dipakai masyarakat masyarakatnya keluh negara ngutang negara ngutang utang dibay...  ,
 (3630, 2))

# Text Stemming Using Sastrawi for Bahasa

In [56]:
def stemming_indonesian_sastrawi(tweet):
    # create an instance of the stemming_indoensian_sastrawi class
    stemmer = StemmerFactory().create_stemmer()

    # Performing Stemming on each element of the series
    stemmed_series = tweet.apply(lambda x: stemmer.stem(x))

    return stemmed_series

In [57]:
df_tweets['cleaned_tweets'] = stemming_indonesian_sastrawi(df_tweets['cleaned_tweets'])
print(df_tweets['cleaned_tweets'].head())
print(df_tweets['cleaned_tweets'].tail())

0                                                 prabowo indonesia harga bangsa asing berita hoax buat kuasa ya rockygerung
1                                                                  batu langka tasbih jokowi hadiah habib luthfi harga mercy
2                                                                                                   jokowi ekonomi indonesia
3                      sumatera selatan asi games dampak ekonomi langsung prediksi capai triliun indonesia maju jokowi hebat
4    negara ngutang bangun infrastruktur pakai masyarakat masyarakat keluh negara ngutang negara ngutang utang bayar nega...
Name: cleaned_tweets, dtype: object
1810                       negarawan sejati bangga depan harga bangsa daulat
1811                                    ceramah damai indonesia ekonomi baik
1812               mari bangun bangsa dukung ekonomi negara jokowi ayo kerja
1813                                        bantu maju ekonomi bangsa jokowi
1814    jokowi ubah pandang eko

In [58]:
df_combine_tweets['tweet'] = stemming_indonesian_sastrawi(df_combine_tweets['tweet'])
print(df_combine_tweets['tweet'].head())
print(df_combine_tweets['tweet'].tail())

0                                                 prabowo indonesia harga bangsa asing berita hoax buat kuasa ya rockygerung
1                                                                  batu langka tasbih jokowi hadiah habib luthfi harga mercy
2                                                                                                   jokowi ekonomi indonesia
3                      sumatera selatan asi games dampak ekonomi langsung prediksi capai triliun indonesia maju jokowi hebat
4    negara ngutang bangun infrastruktur pakai masyarakat masyarakat keluh negara ngutang negara ngutang utang bayar nega...
Name: tweet, dtype: object
1810                      negarawan sejati bangga cadang harga bangsa daulat
1811                                    ceramah damai indonesia ekonomi baik
1812               mari bangun bangsa dukung ekonomi negara jokowi ayo kerja
1813                                        bantu maju ekonomi bangsa jokowi
1814    jokowi malih anggap ekonomi oran

In [63]:
df_combine_tweets = df_combine_tweets.reset_index(drop=True)
df_combine_tweets = shuffle(df_combine_tweets)
df_combine_tweets.shape

(3630, 2)

In [66]:
# drop kolom tweet yang panjang len nya 0
df_combine_tweets = df_combine_tweets[df_combine_tweets['tweet'].str.len() != 0]
print(df_combine_tweets.head(5))
print(df_combine_tweets.shape)

     sentimen  \
1804  positif   
3274  negatif   
1107  negatif   
2490   netral   
2152   netral   

                                                                                                                        tweet  
1804                                                          ketua ada presiden wakil serah gaji rakyat perlu rakyat prabowo  
3274  merdeka dungu otak pakai dasar dismpaikan prabowo ttng ekonomi nyungsep sembako mahal tani susah lpngan kerja sulit ...  
1107                                                       maaf baoak target tumbah kembang tumbuh ekonomi janji jokowi wujud  
2490  coba bicara gin jaman lulung topik fungsi dprd mitra bidang legislatif awas gubernur jalan anggar bayar pajak senang...  
2152                                                                                                     kunci ekonomi rakyat  
(3627, 2)


# Save Cleaning Tweets

## for EDA

In [67]:
len_tweets = pd.Series([len(str(row).split()) for row in df_tweets['cleaned_tweets']])

In [68]:
cleaned_tweets = pd.concat([df_tweets.sentimen, tweets, df_tweets['cleaned_tweets'], len_tweets], axis=1)
cleaned_tweets.to_csv('cleaned_tweets.csv', index=False)

## for Text Text Augmentation, Tokenization, and Vectorization 

In [69]:
df_combine_tweets.to_csv('combined_tweets.csv', index=False)